# ARIMA a base de datos Sunspots

## Limpieza del dataset

In [24]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path

In [25]:
ROOT = Path.cwd().resolve().parent
DATA = ROOT / "Data" / "Sunspots.csv"

In [57]:
df = pd.read_csv(DATA, index_col=0)
df

,Date,Monthly Mean Total Sunspot Number
0,1749-01-31,96.7
1,1749-02-28,104.3
2,1749-03-31,116.7
3,1749-04-30,92.8
4,1749-05-31,141.7
...,...,...
3260,2020-09-30,0.6
3261,2020-10-31,14.4
3262,2020-11-30,34.0
3263,2020-12-31,21.8


Los datos ya parecen ordenamos pero aún así evitando cualquier posible escenario vamos a ordenarlos por Date

In [59]:
df = df.sort_values('Date')

Veamos si tenemos valores nulos

In [27]:
df.isnull().any().any()

np.False_

In [28]:
df.isnull().sum()

Date                                 0
Monthly Mean Total Sunspot Number    0
dtype: int64

Como podemos ver no tenemos columnas con valores NaN

Ahora, veamos si nuestro dataset siempre tiene un mes de diferencia entre registro

In [ ]:
df['Date'] = pd.to_datetime(df['Date']) # Pasaremos la columna Date a formato datetime

In [55]:
diff = df['Date'].dt.to_period('M').astype('int64').diff() # Calculamos la diferencia entre meses

In [56]:
diff #Pareciera que todos los datos tienen un mes de diferencia

0       NaN
1       1.0
2       1.0
3       1.0
4       1.0
       ... 
3260    1.0
3261    1.0
3262    1.0
3263    1.0
3264    1.0
Name: Date, Length: 3265, dtype: float64

In [53]:
es_consecutivo = (diff.dropna() == 1).all()
print(es_consecutivo)
# True -> Todos los valores son 1 lo que implica que tenemos una serie de tiempo limpia
# False -> Hay huecos o fechas repetidas

True


Podemos concluir que tenemos una serie de tiempo con intervalos de tiempo discreta donde la unidad es mes y no hay datos faltantes.

Ahora dividiremos los datos en train y test para la parte de validación

Donde el 90% de los datos irá para train y el 10% para la validación, recordemos que es una serie de tiempo por lo que se cortará de forma lineal y no de forma aleatoria.

In [61]:
cut = int(len(df) * 0.9) # Indice donde empezarán los datos de validación
train = df.iloc[:cut]
valid = df.iloc[cut:]

train.to_csv( ROOT / "Data" /'train.csv', index=False)
valid.to_csv( ROOT / "Data" /'valid.csv', index=False)

In [62]:
cut

2938

Tenemos un total de 3265 registros de manchas solares que son datos desde 1749-01-31 a 2021-01-31 de los cuales tomaremos 2937 datos para construir nuestro modelo y con 328 datos realizaremos la validación de nuestro modelo.